In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import ast 
from sklearn.ensemble import RandomForestRegressor # ML
from sklearn.inspection import permutation_importance # computing feature importance
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:

data = pd.read_csv("bollywooddatasetfinal.csv",encoding='latin-1')
data.head()


,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Movie Budget,Revenue
0,83,2021,165.0,Sport/Drama,7.4,"27,958",Kabir Khan,Ranveer Singh,Deepika Padukone,Hardy Sandhu,2800000000,1937300000
1,#Homecoming,2021,90.0,Drama/ Musical,7.6,335,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,560000000,1230000000
2,102 Not Out,2018,102.0,Comedy/Dram,7.4,"7,112",Umesh Shukla,Amitabh Bachchan,Rishi Kapoor,Jimit Trivedi,350000000,1150000000
3,12 Bulbule,2021,NaN,Comedy/ Drama/ Family,8.9,3659,Jagat Joon,Sonu Sonkar,Sunita Verman,Anubhav Dixit,30000000,60000000
4,13 Tribute of Love,2020,66.0,Drama,5,352,Rahul Tiwari,Rahul Tiwari Adhiyari,Anil Arora,Asima Bhatt,3000000,15000000


In [18]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data['Genre_code'] = lb_make.fit_transform(data['Genre']).astype('int64')
data['Actor1_code'] = lb_make.fit_transform(data['Actor 1']).astype('int64')
data['Director_code'] = lb_make.fit_transform(data['Director']).astype('int64')
data['Revenue'] = pd.to_numeric(data['Revenue'] ,downcast='integer', errors='coerce')
data['Year'] = pd.to_numeric(data['Year'] ,downcast='integer',errors='coerce')
data.head()


,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Movie Budget,Revenue,Genre_code,Actor1_code,Director_code
0,83,2021.0,165.0,Sport/Drama,7.4,"27,958",Kabir Khan,Ranveer Singh,Deepika Padukone,Hardy Sandhu,2800000000,1.937300e+09,111,105,95
1,#Homecoming,2021.0,90.0,Drama/ Musical,7.6,335,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,560000000,1.230000e+09,81,120,200
2,102 Not Out,2018.0,102.0,Comedy/Dram,7.4,"7,112",Umesh Shukla,Amitabh Bachchan,Rishi Kapoor,Jimit Trivedi,350000000,1.150000e+09,53,16,210
3,12 Bulbule,2021.0,NaN,Comedy/ Drama/ Family,8.9,3659,Jagat Joon,Sonu Sonkar,Sunita Verman,Anubhav Dixit,30000000,6.000000e+07,46,133,85
4,13 Tribute of Love,2020.0,66.0,Drama,5,352,Rahul Tiwari,Rahul Tiwari Adhiyari,Anil Arora,Asima Bhatt,3000000,1.500000e+07,71,91,154


In [19]:
# Replacing missing values with 0:
data['Genre_code'] = data['Genre_code'].fillna(0)
data['Actor1_code'] = data['Actor1_code'].fillna(0)
data['Director_code'] = data['Director_code'].fillna(0)
data['Revenue'] = data['Revenue'].fillna(0)
data['Year'] = data['Year'].fillna(0)
data['Movie Budget'] = data['Movie Budget'].replace(0, data['Movie Budget'].median())
#train['budget']= train['budget'].replace(0, train['budget'].median())
data['Duration'] = data['Duration'].fillna(0)
data.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3,Movie Budget,Revenue,Genre_code,Actor1_code,Director_code
0,83,2021.0,165.0,Sport/Drama,7.4,"27,958",Kabir Khan,Ranveer Singh,Deepika Padukone,Hardy Sandhu,2800000000,1.937300e+09,111,105,95
1,#Homecoming,2021.0,90.0,Drama/ Musical,7.6,335,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana,560000000,1.230000e+09,81,120,200
2,102 Not Out,2018.0,102.0,Comedy/Dram,7.4,"7,112",Umesh Shukla,Amitabh Bachchan,Rishi Kapoor,Jimit Trivedi,350000000,1.150000e+09,53,16,210
3,12 Bulbule,2021.0,0.0,Comedy/ Drama/ Family,8.9,3659,Jagat Joon,Sonu Sonkar,Sunita Verman,Anubhav Dixit,30000000,6.000000e+07,46,133,85
4,13 Tribute of Love,2020.0,66.0,Drama,5,352,Rahul Tiwari,Rahul Tiwari Adhiyari,Anil Arora,Asima Bhatt,3000000,1.500000e+07,71,91,154


In [20]:
df = pd.DataFrame(data)
column_names = df[['Year','Movie Budget', 'Genre_code', 'Duration', 'Director_code', 'Actor1_code', 'Revenue']]
data_sklearn = column_names.copy()
data_sklearn.head(10)

,Year,Movie Budget,Genre_code,Duration,Director_code,Actor1_code,Revenue
0,2021.0,2800000000,111,165.0,95,105,1.937300e+09
1,2021.0,560000000,81,90.0,200,120,1.230000e+09
2,2018.0,350000000,53,102.0,210,16,1.150000e+09
3,2021.0,30000000,46,0.0,85,133,6.000000e+07
4,2020.0,3000000,71,66.0,154,91,1.500000e+07
5,1994.0,47500000,19,157.0,214,18,3.490000e+07
6,2019.0,8000000,112,107.0,52,3,2.000000e+05
7,2014.0,320000000,106,149.0,13,23,1.750000e+09
8,2009.0,550000000,45,170.0,160,1,4.000000e+09
9,2016.0,10000000,65,115.0,187,116,5.400000e+06


In [21]:
feature_names1 = ['Year','Movie Budget', 'Genre_code', 'Duration', 'Director_code', 'Actor1_code']
X_train = data_sklearn[feature_names1]

In [7]:
# feature_names2 = ['Year','Movie Budget', 'Genre_code', 'Duration', 'Director_code', 'Actor1_code']
X_test = data_sklearn[feature_names1]
X_test.head()

,Year,Movie Budget,Genre_code,Duration,Director_code,Actor1_code
0,2021.0,56.00,10,90.0,200,120
1,2021.0,3.00,7,0.0,85,133
2,2020.0,0.30,10,66.0,154,91
3,1994.0,4.75,0,157.0,214,18
4,2019.0,0.80,17,107.0,52,3


In [22]:
# Assigning the data corresponding to the target variable

y_train = data_sklearn['Revenue']

In [23]:
# Defining the model 
rf_model = RandomForestRegressor(random_state=1)

# Fitting the model 
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [25]:
y_pred = rf_model.predict(X_test)
# y_pred = y_pred.astype('int64')
pred = pd.DataFrame(y_pred)

In [26]:
from joblib import dump

In [27]:
dump(rf_model,'./../savedModels/model.joblib')

['./../savedModels/model.joblib']

In [ ]:
# release_year = int(input("Enter the release year: "))
# budget = float(input("Enter the budget: "))
# genres = int(input("Enter the genere: "))
# duration = float(input("Enter the movie duration: "))
# director = int(input("Enter the movie director: "))
# actor = int(input("Enter the lead actor: "))

: 

In [ ]:
# result = rf_model.predict([[release_year, budget, genres, duration,
#                  director, actor]])
# print(result.astype('int64'))

: 

In [ ]:
# from sklearn import metrics
# import numpy as np
# print('Mean Absolute Error:' , metrics.mean_absolute_error())

: 